# Numerical PDE II: Spectral Methods

## Burgers' Equation

In the previous lecture, we analyzed finite difference methods for the
linear advection equation,
\begin{align}
  \frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = 0,
\end{align}
and studied their stability and accuracy using von Neumann analysis
and modified equations.

Now, we turn to a nonlinear extension: the Burgers' equation.
By replacing the constant wave speed $c$ with the solution variable
itself, the equation becomes
\begin{align}
  \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0.
\end{align}
This is the inviscid Burgers' equation, a canonical model for
nonlinear advection.

In [ ]:
# Parameters

l  = 1.0   # domain size
dt = 0.01  # time step

nx = 100   # number of spatial points
nt = 20    # number of time steps

In [ ]:
import numpy as np

X, dx = np.linspace(0, l, nx, endpoint=False, retstep=True)  # spatial grid
U0    = np.sin(2*np.pi * X)  # initial condition

In [ ]:
# Forward Time Centered Space (FTCS) scheme

def FTCS(U0, dx, dt, n):
    U = [U0]
    for _ in range(n):
        U0    = U[-1]
        sigma = U0 * dt / dx
        U1    = U0 - sigma * (np.roll(U0,-1) - np.roll(U0,1)) / 2
        U.append(U1)
    return np.array(U)

In [ ]:
UFTCS = FTCS(U0, dx, dt, nt) # numerical solution

In [ ]:
from matplotlib import pyplot as plt

plt.plot(X, U0,              label="Initial Condition")
plt.plot(X, UFTCS[10], '.-', label=f'$t$={10*dt}')
plt.plot(X, UFTCS[14], '.-', label=f'$t$={14*dt}')
plt.plot(X, UFTCS[18], '.-', label=f'$t$={18*dt}')
plt.xlabel('x')
plt.ylabel('u')
plt.legend()

In [ ]:
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML
from tqdm import tqdm

def animate(X, U):
    fig, ax = plt.subplots(1,1)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    frames = []
    for n in tqdm(range(len(U))):
        f = ax.plot(X, U[n], 'C0.-', animated=True)
        frames.append(f)
        plt.close()
    
    return ArtistAnimation(fig, frames, interval=40)

In [ ]:
anim = animate(X, UFTCS)

HTML(anim.to_html5_video())  # display animation
# anim.save('FTCS.mp4')        # save animation

In [ ]:
# HANDSON: Plot other snapshots and study the oscillations.


In [ ]:
# HANDSON: Change the parameters and initial conditions.


The Burgers' equation is important because:
* It demonstrates wave steepening: smooth initial profiles evolve into
  discontinuous shocks.
* It highlights the limitations of finite difference schemes:
  dispersion and oscillations arise near shocks.

As with the linear advection equation, the FTCS scheme is
unconditionally for Burgers' equation.
Even with very small time steps, oscillations grow and the solution
eventually blows up.

The numerical solution initially appears to evolve correctly.
However, once the shock starts to form, the numerical solution quickly
develops high-frequency oscillations near the shock.
These oscillations grow exponentially in time, destroying the
solution.

There is no convergence---the method is unstable.

In [ ]:
# HANDSON: modify the upwind scheme from lecture 9 to solve the
#          Burgers' equation

def upwind(U0, dx, dt, n):
    ...

In [ ]:
dt = 0.01  # time step
nt = 100   # number of time steps

Uupwind = upwind(U0, dx, dt, nt) # numerical solution

In [ ]:
plt.plot(X, U0,                label="Initial Condition")
plt.plot(X, UFTCS[18],   '.-', label="FTCS")
plt.plot(X, Uupwind[18], '.-', label="Upwind")
plt.xlabel('x')
plt.ylabel('u')
plt.legend()

In [ ]:
anim = animate(X, Uupwind)

HTML(anim.to_html5_video())  # display animation
# anim.save('upwind.mp4')      # save animation

In [ ]:
# HANDSON: Plot other snapshots and study the oscillations.


In [ ]:
# HANDSON: Change the parameters and initial conditions.
#          Specifically, try out dt > 0.01.


For the upwind method, the maximum `dt` that support stable solution
is limited by the Courant-Friedrichs-Lewy condition, i.e., at each
grid point, we require the Courant number $\sigma = u \Delta t/\Delta
x \leq 1$.

The scheme captures shocks robustly and monotonically.
The numerical viscosity (implicit by the numerical method) spreads
shocks over several grid cells.
However, the smooth regions damp out over time due to numerical
diffusion.

The convergence rate is first-order everywhere.

Next, let's explore the Lax-Wendroff Scheme.

In [ ]:
# Lax-Wendroff scheme

def LW(U0, dx, dt, n):
    ...

In [ ]:
dt = 0.01  # time step
nt = 100   # number of time steps

ULW = LW(U0, dx, dt, nt) # numerical solution

In [ ]:
plt.plot(X, U0,                label="Initial Condition")
plt.plot(X, UFTCS[18],   '.-', label="FTCS")
plt.plot(X, Uupwind[18], '.-', label="Upwind")
plt.plot(X, ULW[18],     '.-', label="Lax-Wendroff")
plt.xlabel('x')
plt.ylabel('u')
plt.legend()

In [ ]:
anim = animate(X, ULW)

HTML(anim.to_html5_video())  # display animation
# anim.save('LW.mp4')      # save animation

The Lax-Wendroff scheme, similar to upwind scheme, is stable under CFL
condition.

The solution has excellent accuracy for smooth solutions.
However, it has Gibbs-like oscillations appear at shocks.
It is non-monotone but conservative.

The convergence rate is second-order for smooth flows, first-order for
discontinuous ones.

## Weak Solutions

Inviscid Burgers' equation develops discontinuities (shocks) in finite
time, even from smooth initial conditions.
For example, an initial sine wave we saw earlier steepens until the
slope blows up.

This leads to a breakdown of classical solutions.
Namely, the PDE cannot be satisfied pointwise after a discontinuity
forms.
To make sense of solutions beyond shock formation, we introduce the
concept of a weak solution.

Multiply a PDE by a smooth test function $\phi(x,t)$, integrate over
space and time, and integrate by parts:
\begin{align}
  \int_0^\infty \int_{0}^\ell
  \left(u\frac{\partial\phi}{\partial t}
        + \frac{1}{2}u^2\frac{\partial\phi}{\partial x}\right)
  dx dt = 0.
\end{align}

The solution satisfying the above equation is called a weak solution.
Note that this definition still makes sense if $u$ is discontinuous.
Thus, weak solutions allow shocks.

Not all weak solutions are physically relevant.
The entropy condition rules out unphysical "expansion shocks".
For Burgers, this requires that characteristics enter the shock (no
information leaves it).